# Global Coastal Transect Repository (GCTR)

The **Global Coastal Transect Repository (GCTR)** is a comprehensive data repository designed for global coastal analytics. It builds upon the **Global Coastal Transect System (GCTS)**, which provides over 11 million uniformly spaced cross-shore transects at 100-m intervals. These transects serve as standardized reference lines for measuring coastal change, mapping coastal features, and deriving coastal statistics thereof. See Calkoen et al. (2025) "Enabling Coastal Analytics at Planetary Scale" or [this](https://radiantearth.github.io/stac-browser/#/external/coclico.blob.core.windows.net/stac/v1/gcts/collection.json) STAC collection for more details.

Besides transects and administrative boundaries this data repository now includes:
- **Intersection distances to OSM coastlines** at multiple bearings (e.g., 0°, 30°, 330°).
- **Shoreline change rates** from the **ShorelineMonitor's SDS** (see Luijendijk et al., 2018 and [this](https://radiantearth.github.io/stac-browser/#/external/coclico.blob.core.windows.net/stac/v1/shorelinemonitor-shorelines/collection.json) STAC collection for more details).
- A machine-learning-derived **coastal typology**. 

In [ ]:
import os

import dotenv
import fsspec
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
import shapely
from dotenv import load_dotenv
from ipyleaflet import Map, basemaps

from coastpy.stac.utils import read_snapshot

load_dotenv()

# Configure cloud and Dask settings
sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_options = {"account_name": "coclico", "sas_token": sas_token}

coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)
collection = coclico_catalog.get_child("gctr")

In [ ]:
snapshot = read_snapshot(collection, storage_options=storage_options)
snapshot.head()

In [ ]:
from ipyleaflet import Map, basemaps

m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = (43.32, -1.97)
m.zoom = 14
m.layout.height = "800px"
m

In [ ]:
from coastpy.geo.utils import get_region_of_interest_from_map

roi = get_region_of_interest_from_map(m, default_extent=(4.796, 53.108, 5.229, 53.272))
west, south, east, north = roi.geometry.item().bounds

## Fetch data from the database

In [ ]:
import coastpy

db = coastpy.io.STACQueryEngine(
    stac_collection=collection,
    storage_backend="azure",
    # Use columns to filter when you don't need all data;
    columns=[
        "geometry",
        "lon",
        "lat",
        "transect_id",
        "sds:change_rate",
        "class:shore_type",
        "class:coastal_type",
    ],
)

In [ ]:
from coastpy.utils.config import fetch_sas_token

sas_token = fetch_sas_token(sas_token)
df = db.get_data_within_bbox(west, south, east, north, sas_token=sas_token)
print(f"Shape: {df.shape}")
df.head()

In [ ]:
df.groupby(["class:shore_type", "class:coastal_type"])[
    "sds:change_rate"
].mean().sort_values(ascending=True).to_frame()